In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
import librosa
import numpy as np

# Load the Whisper model and processor from Hugging Face
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [6]:
def transcribe_audio(file_path):
    # Load and process the audio file
    audio_data, sampling_rate = librosa.load(file_path, sr=16000)
    audio_input = processor(audio_data, sampling_rate=sampling_rate, return_tensors="pt", language='en').input_features

    # Generate transcription
    with torch.no_grad():
        generated_ids = model.generate(audio_input)
        transcript = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return transcript

# Test the speech-to-text function on an audio file
if __name__ == '__main__':
    audio_file_path = "/Users/sasanksasi/Downloads/project/wizard/audio_sample/harvard.wav"
    
    # Call the transcription function
    transcript = transcribe_audio(audio_file_path)
    
    print("Transcript:", transcript)

Transcript:  The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham. Tacos al pastor are my favorite. A zestful food is the hot cross bun.


In [ ]:
import os
import json
import logging
import re
from typing import Dict, Any, Optional, List
from datetime import datetime
from groq import Groq
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('transcript_analysis.log')
    ]
)

# Initialize Groq API client
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise EnvironmentError("GROQ_API_KEY not found in environment variables")
client = Groq(api_key=api_key)

def clean_text(text: str) -> str:
    """Clean and normalize text"""
    return text.strip() if text else ""

def extract_dates(text: str) -> List[str]:
    """Extract dates from text using regex"""
    date_pattern = r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2}(?:st|nd|rd|th)?,? \d{4})\b'
    return list(set(re.findall(date_pattern, text, re.IGNORECASE)))

def extract_emails(text: str) -> List[str]:
    """Extract email addresses from text"""
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    return list(set(re.findall(email_pattern, text)))

def analyze_transcript_with_groq(transcript: str) -> Dict[str, Any]:
    """Analyze transcript using Groq API with improved error handling"""
    if not transcript:
        return {"error": "Empty transcript provided"}

    prompt = f"""
    Analyze this transcript and provide only a JSON response with this structure:
    {{
        "transcript": "full transcript text",
        "summary": "summary of the transcript",
        "key_points": ["point 1", "point 2, ..."],
        "dates": ["date/time reference 1", "date/time reference 2, ..."],
        "emails": ["email1@domain.com"],
        "action_items": ["action 1", "action 2, ..."]
    }}
    
    Transcript: {transcript}
    
    Important: Return ONLY valid JSON, no markdown or additional text.
    """
    
    try:
        logging.info("Sending request to Groq API")
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a JSON-only response bot. Always respond with valid JSON matching the specified structure. No markdown, no explanations, just JSON."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model="llama3-8b-8192",
            temperature=0.1
        )
        
        # Clean and parse response
        response = chat_completion.choices[0].message.content.strip()
        response = re.sub(r'^[^{]*', '', response)  # Remove anything before first {
        response = re.sub(r'[^}]*$', '', response)  # Remove anything after last }
        
        try:
            parsed_response = json.loads(response)
            
            # Add extracted dates and emails
            extracted_dates = extract_dates(transcript)
            extracted_emails = extract_emails(transcript)
            
            if extracted_dates:
                parsed_response["dates"] = list(set(parsed_response.get("dates", []) + extracted_dates))
            if extracted_emails:
                parsed_response["emails"] = list(set(parsed_response.get("emails", []) + extracted_emails))
            
            return parsed_response
            
        except json.JSONDecodeError as e:
            logging.error(f"JSON parsing error: {str(e)}")
            return {
                "error": "JSON parsing error",
                "details": str(e),
                "raw_response": response
            }
            
    except Exception as e:
        logging.error(f"Groq API error: {str(e)}")
        return {"error": f"API error: {str(e)}"}

def validate_response(response: Dict[str, Any]) -> bool:
    """Validate response structure and types"""
    if "error" in response:
        return False
        
    required_fields = {
        "transcript": str,
        "summary": str,
        "key_points": list,
        "dates": list,
        "emails": list,
        "action_items": list
    }
    
    try:
        for field, field_type in required_fields.items():
            if field not in response or not isinstance(response[field], field_type):
                return False
        return True
    except Exception:
        return False

def format_and_print_analysis(analysis: Dict[str, Any]) -> None:
    """Format and print analysis results"""
    if "error" in analysis:
        print("\n❌ Error in Analysis:")
        print(f"   {analysis['error']}")
        if "raw_response" in analysis:
            print("\nRaw Response:")
            print(analysis['raw_response'])
        return

    print("\n📊 === Transcript Analysis ===\n")
    
    sections = {
        "transcript": "📝 Full Transcript",
        "summary": "📌 Summary",
        "key_points": "🔑 Key Points",
        "dates": "📅 Dates Mentioned",
        "emails": "📧 Email Addresses",
        "action_items": "✅ Action Items"
    }
    
    for key, title in sections.items():
        print(f"\n{title}")
        print("─" * len(title))
        value = analysis.get(key, [])
        if isinstance(value, list):
            if value:
                for item in value:
                    print(f"• {item}")
            else:
                print("None found")
        else:
            print(value if value else "None provided")

def save_analysis_to_file(analysis: Dict[str, Any], output_path: str) -> None:
    """Save analysis results to JSON file"""
    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(analysis, f, indent=2, ensure_ascii=False)
        logging.info(f"Analysis saved to: {output_path}")
        print(f"\n💾 Analysis saved to: {output_path}")
    except Exception as e:
        logging.error(f"Error saving analysis: {str(e)}")
        print(f"\n❌ Error saving analysis: {str(e)}")

def process_transcript(transcript_text: str, output_file: Optional[str] = None) -> Dict[str, Any]:
    """Process transcript and save results"""
    try:
        # Clean input
        transcript_text = clean_text(transcript_text)
        if not transcript_text:
            return {"error": "Empty transcript provided"}
            
        # Get analysis
        analysis = analyze_transcript_with_groq(transcript_text)
        
        # Validate response
        if not validate_response(analysis):
            return {"error": "Invalid response structure", "raw_response": analysis}
        
        # Print analysis
        format_and_print_analysis(analysis)
        
        # Save if path provided
        if output_file:
            save_analysis_to_file(analysis, output_file)
        
        return analysis
        
    except Exception as e:
        logging.error(f"Error processing transcript: {str(e)}")
        return {"error": f"Processing error: {str(e)}"}

# Update the main execution block
if __name__ == "__main__":
    # Test transcript
    # test_transcript = """
    # Hello everyone, welcome to our weekly meeting on March 15th, 2024.
    
    # Our main action items from last week:
    # 1. John needs to send the report to sarah.smith@company.com
    # 2. Meeting with clients scheduled for March 20th at 2 PM
    # 3. Project deadline is set for April 1st, 2024
    
    # Please contact support@company.com for any technical issues.
    # Next meeting is scheduled for March 22nd, 2024.
    # """
    
    try:
        # Create output directory in current working directory
        output_dir = os.path.join(os.getcwd(), "output")
        os.makedirs(output_dir, exist_ok=True)
        
        # Generate output path with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_path = os.path.join(
            output_dir,
            f"transcript_analysis_{timestamp}.json"
        )
        
        logging.info(f"Processing transcript and saving to: {output_path}")
        results = process_transcript(transcript, output_path)
        
        # Print status
        if "error" not in results:
            print("\n✅ Analysis completed successfully!")
            logging.info("Analysis completed successfully")
        else:
            print(f"\n❌ Analysis failed: {results['error']}")
            logging.error(f"Analysis failed: {results['error']}")
            
    except Exception as e:
        logging.error(f"Fatal error: {str(e)}")
        print(f"\n❌ Fatal error: {str(e)}")

2025-02-24 15:18:41,962 - INFO - Processing transcript and saving to: /Users/sasanksasi/Downloads/project/wizard/output/transcript_analysis_20250224_151841.json
2025-02-24 15:18:41,963 - INFO - Sending request to Groq API
2025-02-24 15:18:42,455 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-24 15:18:42,463 - INFO - Analysis saved to: /Users/sasanksasi/Downloads/project/wizard/output/transcript_analysis_20250224_151841.json
2025-02-24 15:18:42,463 - INFO - Analysis completed successfully



📊 === Transcript Analysis ===


📝 Full Transcript
─────────────────
The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham. Tacos al pastor are my favorite. A zestful food is the hot cross bun.

📌 Summary
─────────
The speaker discusses the effects of heat on odors and their favorite foods.

🔑 Key Points
────────────
• heat brings out odors
• cold dip restores health

📅 Dates Mentioned
─────────────────
None found

📧 Email Addresses
─────────────────
None found

✅ Action Items
──────────────
None found

💾 Analysis saved to: /Users/sasanksasi/Downloads/project/wizard/output/transcript_analysis_20250224_151841.json

✅ Analysis completed successfully!
